In [ ]:
import numpy as np
import pandas as pd
import allel
import random
import os
import sys
import snakemake

### pi is 0.0001 since length_chr5 = 1702174 * 0.0001 = 170
pi =  0.001
beta = 5

######

### calculation of the number of contributing positions and effect sizes of each ###################
file_p = allel.read_vcf('chr5_grenenet.vcf', fields='variants/POS')
## get all the pos from the vcf file
file_p = allel.read_vcf(vcf_file, fields='variants/POS')
pos = file_p['variants/POS']
n_pos = len(pos)
n_pos_contr =round(pi*n_pos)
## randomly select the positions that are going to contribute 
pos_contr = random.sample(pos.tolist(), k=n_pos_contr)
## now im gonna sample the number of nps contributing from a normal distribution with mean 0 and sd defined by beta 
print('For this run the value of pi is ' + str(pi) + ' and the value of beta is ' + str(beta) + ' based on the number of positions being ' + str(n_pos) + ' the number of contributing SNPs is ' + str(n_pos_contr))
mu, sigma = 0,beta # mean and standard deviation
effect_sizes = np.random.normal(mu, sigma, n_pos_contr)
effect_sizes = np.round(effect_sizes, 4)
## now i have to create a bed file to add this infromation to the vcf file 
## create dataframe of the contributing pos 
## to caluclate the positions FROM in a bed file, the fist value is not included
positions_from = [i-1 for i in pos]
# Define the positions and values
chromosome = [chr_number] * len(pos)
# Create a DataFrame from the positions and values
all_pos = pd.DataFrame({'chromosome':chromosome,'positions_from': positions_from, 'positions_to': pos})
contrib_pos = pd.DataFrame({'positions_to': pos_contr, 'sel_coef':effect_sizes})
# merge and fillna with 0 since that is the selection coefficient for all the non contributing snps 
all_pos = all_pos.merge(contrib_pos, left_on= 'positions_to', right_on= 'positions_to' ,how='left').fillna(0)
# Write the DataFrame to a BED file



In [ ]:
## output 
all_pos.to_csv(bed_sc, sep='\t', header=False, index=False) # selection_coef_chr5.bed
print('Finished creating bed file to annotate vcf with effect sizes at each contributing position')